In [ ]:
import tensorflow as tf
import numpy as np
import nltk
import json
import io
import pandas as pd
import pickle
import random

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving restaurant1.json to restaurant1.json


In [ ]:
data = pd.read_json(io.StringIO(uploaded.get('restaurant1.json').decode('utf-8')))
data

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'Hey', ..."
1,"{'tag': 'book_table', 'patterns': ['Book a tab..."
2,"{'tag': 'available_tables', 'patterns': ['How ..."
3,"{'tag': 'goodbye', 'patterns': ['cya', 'I will..."
4,"{'tag': 'identity', 'patterns': ['what is your..."
5,"{'tag': 'hours', 'patterns': ['when are you gu..."
6,"{'tag': 'menu', 'patterns': ['Id like to order..."
7,"{'tag': 'contact', 'patterns': ['contact infor..."
8,"{'tag': 'address', 'patterns': ['what is the l..."
9,"{'tag': 'positive_feedback', 'patterns': ['the..."


In [ ]:
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()

In [ ]:
nltk.download('punkt')

try:
  with open('data.pickle','rb') as f:
    words,labels,training,output= pickle.load()

except:
  words=[]
  doc_patterns=[]
  doc_tag=[]
  labels=[]

  for intent in data['intents']:
    for pattern in intent['patterns']:
      wrd= nltk.word_tokenize(pattern)
      words.extend(wrd)
      doc_patterns.append(wrd)
      doc_tag.append(intent['tag'])
    
    if intent['tag']  not in labels:
      labels.append(intent['tag'])  

  words= [stemmer.stem(w.lower()) for w in words if w != '?']
  words= sorted(list(set(words)))
  labels= sorted(labels)

  training=[]
  output=[]
  out_empty=[0 for _ in range(len(labels))]

  for x,doc in enumerate(doc_patterns):
    bag=[]

    wrds=[stemmer.stem(w.lower()) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
  
    o_row= list(out_empty)
    o_row[labels.index(doc_tag[x])]=1

    training.append(bag)
    output.append(o_row)

  training= np.array(training)
  output= np.array(output)

  with open('data.pickle','wb') as f:
    pickle.dump((words,labels,training,output),f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#model1 (Neural Network)
model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Dense(16, input_shape=(len(training[0]),), activation='relu'))
model1.add(tf.keras.layers.Dense(16, activation='relu'))
model1.add(tf.keras.layers.Dense(16, activation='relu'))
model1.add(tf.keras.layers.Dense(len(output[0]), activation='softmax'))

In [ ]:
model1.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                3808      
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 18)                306       
                                                                 
Total params: 4,658
Trainable params: 4,658
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.fit(training,output,epochs=100,batch_size=8)

Epoch 1/100
27/27 [==============================] - 0s 2ms/step - loss: 2.8854 - accuracy: 0.0619
Epoch 2/100
27/27 [==============================] - 0s 2ms/step - loss: 2.8644 - accuracy: 0.1238
Epoch 3/100
27/27 [==============================] - 0s 2ms/step - loss: 2.8395 - accuracy: 0.1857
Epoch 4/100
27/27 [==============================] - 0s 2ms/step - loss: 2.7999 - accuracy: 0.2286
Epoch 5/100
27/27 [==============================] - 0s 2ms/step - loss: 2.7427 - accuracy: 0.2286
Epoch 6/100
27/27 [==============================] - 0s 2ms/step - loss: 2.6627 - accuracy: 0.2333
Epoch 7/100
27/27 [==============================] - 0s 2ms/step - loss: 2.5523 - accuracy: 0.2333
Epoch 8/100
27/27 [==============================] - 0s 2ms/step - loss: 2.4187 - accuracy: 0.2571
Epoch 9/100
27/27 [==============================] - 0s 2ms/step - loss: 2.2624 - accuracy: 0.3190
Epoch 10/100
27/27 [==============================] - 0s 2ms/step - loss: 2.0918 - accuracy: 0.3714
Epoch 11/

In [ ]:
#model2 (LSTM)



In [ ]:
def bag_of_words(sent,words):
  bag=[0 for _ in range(len(words))]

  s_words= nltk.word_tokenize(sent)
  s_words= [stemmer.stem(wd.lower()) for wd in s_words]

  for s in s_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
  
  return np.array(bag)

In [30]:
def chat():
  print('Chatbot is available(type quit to stop)')

  while True:
    inp= input('You: ')
    if inp.lower()== 'quit':
      break
    
    bag=bag_of_words(inp,words)
    results = model1.predict(np.reshape([bag], (1, 237)))[0]
    results_index = np.argmax(results)
    
    tag= labels[results_index]

    if results[results_index] >= 0.5:

       for tg in data['intents']:
         if tg['tag']==tag:
           response= tg['responses']
   
       print(random.choice(response))

    else:
      print("I didn't that, get please try again.")
  
chat()

Chatbot is available(type quit to stop)
1/1 [==============================] - 0s 60ms/step
Hi! I'm Restrobot. How may I assist you today?
1/1 [==============================] - 0s 16ms/step

1/1 [==============================] - 0s 17ms/step
Hi! I'm Restrobot. How may I assist you today?
1/1 [==============================] - 0s 18ms/step

1/1 [==============================] - 0s 17ms/step

1/1 [==============================] - 0s 17ms/step
You can call me Restrobot.
1/1 [==============================] - 0s 18ms/step
We are open 10am-12am Monday-Friday!
1/1 [==============================] - 0s 17ms/step

You: bye
1/1 [==============================] - 0s 19ms/step
Hope to see you again soon! Goodbye!


KeyboardInterrupt: ignored